<a href="https://colab.research.google.com/github/santiPP/-YOPROGRAMO_SANTIPP/blob/main/AUTO_CHECK_DEVO_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 66.7 MB/s eta 0:00:00


In [8]:
import pandas as pd
import re
import pdfplumber
from collections import defaultdict

In [9]:
# prompt: extract information from excel '/content/drive/My Drive/Colab Notebooks/TCC CONSOLIDE TR03 2025.xlsm'

# Assuming the relevant data is in the first sheet. Change 'Sheet1' if needed.
try:
  df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/TCC CONSOLIDE TR03 2025.xlsm', sheet_name='Result')
  # Print some info about the dataframe to verify the import
  print(df.head())  # Display first 5 rows
  print(df.info())  # Display column data types and non-null values

  # Example: Accessing specific columns or rows.
  # Replace 'Column Name' with the actual name of the column.
  # print(df['Column Name'])
  # print(df.loc[0]) # Access the first row

except FileNotFoundError:
  print("Error: File not found. Check the file path.")
except Exception as e:
  print(f"An error occurred: {e}")

  # Extract the 'PART NUMBER' column.
part_numbers_excel = df['PART NUMBER']

# Extract the 'SPECIFICATIONS' column.
specifications_excel = df['SPECIFICATIONS']

# Print the first 5 elements of each variable to verify.
print(part_numbers_excel.head())
print(specifications_excel.head())

  PART NUMBER                                     SPECIFICATIONS
0  240140229V  FDD/PRCMP/DISEL/RHDD/ACD00,HTSY0/CLKS0/NOSDL,S...
1  240144291V  FDD/PRCMP/DISEL/LHDG/ACD00,HTSY0/CLK00,CLKS0/N...
2  240140299S  FDD/NOVAD/ELECX/LHDG/ACD00,ACD02/CLK00,CLKS0/N...
3  240147846S  FDD/NOVAD/ELECX/LHDG/ACD00,ACD02/CLK00,CLKS0/N...
4  240148966V  FDD/NOVAD,PRCMP/DISEL/LHDG/ACD00,ACD02,HTSY0/C...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   PART NUMBER     115 non-null    object
 1   SPECIFICATIONS  115 non-null    object
dtypes: object(2)
memory usage: 1.9+ KB
None
0    240140229V
1    240144291V
2    240140299S
3    240147846S
4    240148966V
Name: PART NUMBER, dtype: object
0    FDD/PRCMP/DISEL/RHDD/ACD00,HTSY0/CLKS0/NOSDL,S...
1    FDD/PRCMP/DISEL/LHDG/ACD00,HTSY0/CLK00,CLKS0/N...
2    FDD/NOVAD/ELECX/LHDG/ACD00,ACD02/CLK00,CLKS0/N...
3    FD

In [10]:
# prompt: extract part numbers and specifications from pdf '/content/drive/My Drive/Colab Notebooks/FHS898 NEW - TO CHECK - PLANCHER.pdf'. Specifications are codes_list = ["FDD", "EDE", "HDE", "UDE", "L2N", "L3N", "L4L", "13BUS", "15BUS", "16BUS", "ADBUS", "CRWVN", "BGVL0", "DMPER", "FLTBD", "NOVAD", "PRCMP", "DISEL", "ELECX", "LHDG", "RHDD", "SCOLD", "TEMP", "FWDRV", "RWDRV", "NOPWK", "PWRTK", "ACD00", "ACD02", "HTSY0", "HTWSN", "NHTWS", "HTRWI", "NHTRW", "NORWI", "CLK00", "CLKS0", "FFGL0", "NOFFG", "NOSDL", "SDLGT", "NOTCG", "TCGRA", "FAB00", "FAB02", "LPAFW", "LPANW", "LPAOW", "LSLFW", "LSLNW", "LSLOW", "RPAFW", "RPANW", "RPAOW", "RSRFW", "RSRNW", "RSROW", "NORSO", "RSWNW", "NOTPR", "TPRM0", "NOSAB", "SABG0", "SABG2", "FMPBK", "FPASS", "NOFPA", "ALAWP", "ALMWP", "DB4C0", "DK5C0", "DK7C0", "EQHYD", "SAN272", "FWL1T", "FWLPW", "HTS02", "NHTS0", "ASST0", "ASST2", "ASST3", "ASST4", "ASST5", "ASST6", "ASST7", "NOAST", "NA484", "NA485", "RA440", "RA44A", "RA484", "RA486", "SRA12", "SRA13", "1EVGB", "6MAGB", "9ATGB", "RMLT0", "RMLT2", "ASOB6", "ASOC0", "ASOC2", "ASOC3", "ASOC4", "ASOC5", "ASOC7", "ASOC8", "DBWIA", "NODBW", "ALAR3", "NOATD", "CHGS0", "CHGS2", "CHGS3", "CHGS4", "CHGS6", "CHGS7", "CHGS8", "NOCHG", "NOWAD", "WIAD2", "WIAD3", "WIAD4", "WIADA", "LULI0", "LULI2", "NOLUL", "NOWBI", "WBRIN", "ABCXL", "NOCXL", "DBLTI", "NODBT", "E03B", "E06V", "E06W", "E06X", "E07R", "ELC1", "NORDL", "RRDFL", "THMPW", "TPRWR", "TWBPW", "TWHPW", "AHTPA", "NOAHT", "NO938", "V2L00", "V2X02", "HYB00", "HYB02", "NOHYB", "WRAN2", "WRANT", "NOA44", "NORCD", "RCDEV", "NOREW", "REXWI", "NCALL", "RCALL", "BSD00", "BSD07", "BSD08", "NOBSD", "ITP15", "ITPK0", "NOITP", "ANTAC", "NANTA"]

# Initialization of the variables for the PDF file.
part_numbers_pdf = []
specifications_pdf = []

def extract_part_numbers(text):
    """Extracts part numbers from the given text."""
    pattern = r"(\d{9}[A-Za-z]*)"  # Adjust pattern if needed
    return re.findall(pattern, text)

def extract_specifications(text, codes_list):
    """Extracts specifications from the given text."""
    pattern = r'\b(' + '|'.join(re.escape(code) for code in codes_list) + r')\b'
    return re.findall(pattern, text)

codes_list = ["FDD", "EDE", "HDE", "UDE", "L2N", "L3N", "L4L", "13BUS", "15BUS", "16BUS", "ADBUS", "CRWVN", "BGVL0", "DMPER", "FLTBD", "NOVAD", "PRCMP", "DISEL", "ELECX", "LHDG", "RHDD", "SCOLD", "TEMP", "FWDRV", "RWDRV", "NOPWK", "PWRTK", "ACD00", "ACD02", "HTSY0", "HTWSN", "NHTWS", "HTRWI", "NHTRW", "NORWI", "CLK00", "CLKS0", "FFGL0", "NOFFG", "NOSDL", "SDLGT", "NOTCG", "TCGRA", "FAB00", "FAB02", "LPAFW", "LPANW", "LPAOW", "LSLFW", "LSLNW", "LSLOW", "RPAFW", "RPANW", "RPAOW", "RSRFW", "RSRNW", "RSROW", "NORSO", "RSWNW", "NOTPR", "TPRM0", "NOSAB", "SABG0", "SABG2", "FMPBK", "FPASS", "NOFPA", "ALAWP", "ALMWP", "DB4C0", "DK5C0", "DK7C0", "EQHYD", "SAN272", "FWL1T", "FWLPW", "HTS02", "NHTS0", "ASST0", "ASST2", "ASST3", "ASST4", "ASST5", "ASST6", "ASST7", "NOAST", "NA484", "NA485", "RA440", "RA44A", "RA484", "RA486", "SRA12", "SRA13", "1EVGB", "6MAGB", "9ATGB", "RMLT0", "RMLT2", "ASOB6", "ASOC0", "ASOC2", "ASOC3", "ASOC4", "ASOC5", "ASOC7", "ASOC8", "DBWIA", "NODBW", "ALAR3", "NOATD", "CHGS0", "CHGS2", "CHGS3", "CHGS4", "CHGS6", "CHGS7", "CHGS8", "NOCHG", "NOWAD", "WIAD2", "WIAD3", "WIAD4", "WIADA", "LULI0", "LULI2", "NOLUL", "NOWBI", "WBRIN", "ABCXL", "NOCXL", "DBLTI", "NODBT", "E03B", "E06V", "E06W", "E06X", "E07R", "ELC1", "NORDL", "RRDFL", "THMPW", "TPRWR", "TWBPW", "TWHPW", "AHTPA", "NOAHT", "NO938", "V2L00", "V2X02", "HYB00", "HYB02", "NOHYB", "WRAN2", "WRANT", "NOA44", "NORCD", "RCDEV", "NOREW", "REXWI", "NCALL", "RCALL", "BSD00", "BSD07", "BSD08", "NOBSD", "ITP15", "ITPK0", "NOITP", "ANTAC", "NANTA"]

pdf_path = '/content/drive/My Drive/Colab Notebooks/FHS898 - PLANCHER.pdf'

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        part_numbers_pdf = extract_part_numbers(text)
        specifications_pdf = extract_specifications(text, codes_list)

        print("Part Numbers DEVO:", part_numbers_pdf)
        print("Specifications DEVO:", specifications_pdf)

Part Numbers DEVO: ['000581495S']
Specifications DEVO: ['FDD', 'FDD', 'FDD', 'FDD']
Part Numbers DEVO: ['240140229V', '240142215V', '240143096V']
Specifications DEVO: ['NOAST', 'ELECX', 'NOAST', 'ELECX', 'BSD00', 'NOAST', 'ELECX']
Part Numbers DEVO: ['240143796V', '240144291V', '240144768V', '240145258V']
Specifications DEVO: []
Part Numbers DEVO: ['240145541V', '240145911V', '240146295V']
Specifications DEVO: []
Part Numbers DEVO: ['240146572V', '240147203V', '240147227V', '240147368V']
Specifications DEVO: []
Part Numbers DEVO: ['240147941V', '240148547V', '240148699V']
Specifications DEVO: []
Part Numbers DEVO: ['240148966V', '240149014V', '240144920S']
Specifications DEVO: ['FDD', '16BUS', 'ADBUS', 'CRWVN', 'DISEL', 'LHDG', 'LPAFW', 'FDD', '16BUS', 'ADBUS', 'CRWVN', 'DISEL', 'LHDG', 'LPAFW', 'FDD', '16BUS', 'ADBUS', 'LPAOW', 'RSRFW', 'RSROW', 'NOSAB', 'ASST0', 'ASST2', 'LPAOW', 'RSRFW', 'RSROW', 'NOSAB', 'ASST0', 'ASST2', 'RSROW', 'NOSAB', 'ASST3', 'NOCHG', 'NORCD', 'EQHYD', 'V2L00

In [11]:
# prompt: save part_numbers_pdf and specifications_pdf as it changes as a list

# Initialize lists to store the changing data
part_numbers_pdf_history = []
specifications_pdf_history = []

def extract_part_numbers(text):
    """Extracts part numbers from the given text."""
    pattern = r"(\d{9}[A-Za-z]*)"  # Adjust pattern if needed
    return re.findall(pattern, text)

def extract_specifications(text, codes_list):
    """Extracts specifications from the given text."""
    pattern = r'\b(' + '|'.join(re.escape(code) for code in codes_list) + r')\b'
    return re.findall(pattern, text)

# ... (rest of your existing code)

pdf_path = '/content/drive/My Drive/Colab Notebooks/FHS898 - PLANCHER.pdf'

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        current_part_numbers_pdf = extract_part_numbers(text)
        current_specifications_pdf = extract_specifications(text, codes_list)

        # Append the current values to the history lists
        part_numbers_pdf_history.append(current_part_numbers_pdf)
        specifications_pdf_history.append(current_specifications_pdf)

        print("Part Numbers DEVO:", current_part_numbers_pdf)
        print("Specifications DEVO:", current_specifications_pdf)

# Now you have the history of part numbers and specifications
print("\nPart Numbers History:")
for i, part_numbers in enumerate(part_numbers_pdf_history):
    print(f"Iteration {i+1}: {part_numbers}")

print("\nSpecifications History:")
for i, specifications in enumerate(specifications_pdf_history):
    print(f"Iteration {i+1}: {specifications}")

# Now I replace the variable Part Numbers PDF and Specifications PDF with Part Numbers PDF History and Specifications PDF History

part_numbers_pdf = part_numbers_pdf_history
specifications_pdf = specifications_pdf_history

Part Numbers DEVO: ['000581495S']
Specifications DEVO: ['FDD', 'FDD', 'FDD', 'FDD']
Part Numbers DEVO: ['240140229V', '240142215V', '240143096V']
Specifications DEVO: ['NOAST', 'ELECX', 'NOAST', 'ELECX', 'BSD00', 'NOAST', 'ELECX']
Part Numbers DEVO: ['240143796V', '240144291V', '240144768V', '240145258V']
Specifications DEVO: []
Part Numbers DEVO: ['240145541V', '240145911V', '240146295V']
Specifications DEVO: []
Part Numbers DEVO: ['240146572V', '240147203V', '240147227V', '240147368V']
Specifications DEVO: []
Part Numbers DEVO: ['240147941V', '240148547V', '240148699V']
Specifications DEVO: []
Part Numbers DEVO: ['240148966V', '240149014V', '240144920S']
Specifications DEVO: ['FDD', '16BUS', 'ADBUS', 'CRWVN', 'DISEL', 'LHDG', 'LPAFW', 'FDD', '16BUS', 'ADBUS', 'CRWVN', 'DISEL', 'LHDG', 'LPAFW', 'FDD', '16BUS', 'ADBUS', 'LPAOW', 'RSRFW', 'RSROW', 'NOSAB', 'ASST0', 'ASST2', 'LPAOW', 'RSRFW', 'RSROW', 'NOSAB', 'ASST0', 'ASST2', 'RSROW', 'NOSAB', 'ASST3', 'NOCHG', 'NORCD', 'EQHYD', 'V2L00

In [18]:
# prompt: compare specifications between the excel file and the pdf file for the same part number and print the differences of specifications. Also, give the option to dismiss specifications from differences, if all the specifications in one group are available

# ... (Your existing code)

def compare_specifications(excel_data, pdf_data):
    """Compares specifications between Excel and PDF data."""
    differences = defaultdict(list)

    for index, row in excel_data.iterrows():
        part_number = row['PART NUMBER']
        excel_specs = set(str(row['SPECIFICATIONS']).split(','))  # Convert to set for efficient comparison

        # Find matching part numbers in PDF data
        for i, pdf_part_numbers in enumerate(part_numbers_pdf):
            if part_number in pdf_part_numbers:
                pdf_specs = set(specifications_pdf[i])

                # Find differences
                missing_in_pdf = excel_specs - pdf_specs
                extra_in_pdf = pdf_specs - excel_specs

                if missing_in_pdf or extra_in_pdf:
                    differences[part_number].append({
                        'source': 'Excel',
                        'specs': list(excel_specs),
                        'missing': list(missing_in_pdf),
                        'extra': list(extra_in_pdf)
                        })

                    differences[part_number].append({
                        'source': 'PDF',
                        'specs': list(pdf_specs)
                    })

    return differences


def dismiss_specifications(differences, dismissal_criteria):
  """Dismisses specifications from differences based on criteria."""
  dismissed_differences = {}

  for part_number, entries in differences.items():
    # Assuming dismissal_criteria is a set of specs to be dismissed
    dismissed_specs = set(dismissal_criteria)

    filtered_entries = []
    for entry in entries:
      #remove dismissed specs from both excel and pdf data
      filtered_specs = [spec for spec in entry.get('specs', []) if spec not in dismissed_specs]
      if filtered_specs: # add only if the specs list is not empty
        entry['specs'] = filtered_specs
        filtered_entries.append(entry)

    if filtered_entries: # add only if filtered entries is not empty
        dismissed_differences[part_number] = filtered_entries

  return dismissed_differences

# Example Usage
differences = compare_specifications(df, df) # replace second df with processed pdf data.
print("Differences:")
for part_number, diff_data in differences.items():
    print(f"Part Number: {part_number}")
    for data in diff_data:
      print(f"  Source: {data['source']}")
      print(f"  Specs: {', '.join(data['specs'])}")
      if 'missing' in data:
        print(f"  Missing in {data['source']}: {', '.join(data['missing'])}")
      if 'extra' in data:
        print(f"  Extra in {data['source']}: {', '.join(data['extra'])}")
      print("-" * 20)


dismissal_criteria = ["FDD", "EDE"] #replace with your dismissal criteria list
dismissed_differences = dismiss_specifications(differences, dismissal_criteria)

print("\nDifferences after dismissal:")
for part_number, diff_data in dismissed_differences.items():
    print(f"Part Number: {part_number}")
    for data in diff_data:
        print(f"  Source: {data['source']}")
        print(f"  Specs: {', '.join(data.get('specs', []))}") #handle case where specs might be missing

        if 'missing' in data:
            print(f"  Missing in {data['source']}: {', '.join(data['missing'])}")
        if 'extra' in data:
            print(f"  Extra in {data['source']}: {', '.join(data['extra'])}")
        print("-" * 20)


Differences:
Part Number: 240140229V
  Source: Excel
  Specs: ITPK0, RSRFW, RSRNW/NOSAB/FPASS/SAN272/FWL1T, WIADA/NO938/RCDEV/BSD00, FWLPW/NHTS0/NOAST/NOCHG/NOWAD, HTSY0/CLKS0/NOSDL, SDLGT/NOTCG/LPAFW, NOITP/NANTA/, LPANW, BSD08, FDD/PRCMP/DISEL/RHDD/ACD00, NOBSD/ITP15, LSLNW/RPAFW/RPANW, LSLFW, WIAD4
  Missing in Excel: ITPK0, RSRNW/NOSAB/FPASS/SAN272/FWL1T, RSRFW, WIADA/NO938/RCDEV/BSD00, FWLPW/NHTS0/NOAST/NOCHG/NOWAD, HTSY0/CLKS0/NOSDL, NOITP/NANTA/, LPANW, BSD08, FDD/PRCMP/DISEL/RHDD/ACD00, NOBSD/ITP15, LSLNW/RPAFW/RPANW, SDLGT/NOTCG/LPAFW, LSLFW, WIAD4
  Extra in Excel: NOAST, BSD00, ELECX
--------------------
  Source: PDF
  Specs: BSD00, NOAST, ELECX
--------------------
  Source: Excel
  Specs: ITPK0, RSRFW, RSRNW/NOSAB/FPASS/SAN272/FWL1T, WIADA/NO938/RCDEV/BSD00, FWLPW/NHTS0/NOAST/NOCHG/NOWAD, HTSY0/CLKS0/NOSDL, SDLGT/NOTCG/LPAFW, NOITP/NANTA/, LPANW, BSD08, FDD/PRCMP/DISEL/RHDD/ACD00, NOBSD/ITP15, LSLNW/RPAFW/RPANW, LSLFW, WIAD4
  Missing in Excel: ITPK0, RSRNW/NOSAB/FPASS/SA

In [19]:
# # prompt: compare specifications between the excel file and the pdf file for the same part number and print the differences. Also, give the option to dismiss specifications from differences, if all the specifications in one group are available

# # ... (Your existing code)

# def compare_specifications(excel_specs, pdf_specs):
#     """Compares specifications from Excel and PDF, handling potential differences."""
#     excel_set = set(excel_specs)
#     pdf_set = set(pdf_specs)

#     differences = {
#         "only_in_excel": excel_set - pdf_set,
#         "only_in_pdf": pdf_set - excel_set,
#     }

#     return differences


# # Process each part number
# for index, part_number_excel in enumerate(part_numbers_excel):
#     print(f"\nComparing specifications for part number: {part_number_excel}")

#     # Find corresponding part number in the PDF data
#     # Assuming part number is present in the PDF data, if not adapt logic below.
#     for i in range(len(part_numbers_pdf)):
#       if str(part_number_excel) in part_numbers_pdf[i]:  # Convert to string for comparison.
#         differences = compare_specifications(specifications_excel[index], specifications_pdf[i])
#         if differences["only_in_excel"] or differences["only_in_pdf"]:
#             print("Differences found:")
#             if differences["only_in_excel"]:
#                 print("  Specifications only in Excel:", differences["only_in_excel"])
#             if differences["only_in_pdf"]:
#                 print("  Specifications only in PDF:", differences["only_in_pdf"])
#         else:
#             print("No differences found for this part number.")
#         break # Exit inner loop after finding the part number in PDF
#     else:
#       print(f"Part number '{part_number_excel}' not found in PDF data.")




Comparing specifications for part number: 240140229V
Differences found:
  Specifications only in Excel: {'A', 'H', 'W', '1', 'T', 'S', 'F', '2', 'G', '8', 'B', '3', 'I', 'O', 'V', 'L', '/', '0', 'C', 'E', 'N', 'M', ',', '5', 'D', '7', '9', '4', 'R', 'P', 'Y', 'K'}
  Specifications only in PDF: {'NOAST', 'BSD00', 'ELECX'}

Comparing specifications for part number: 240144291V
Differences found:
  Specifications only in Excel: {'A', 'H', 'W', '1', 'T', 'S', 'F', '2', 'G', '8', 'B', '3', 'I', 'O', 'V', 'L', '/', '0', 'C', 'E', 'M', ',', 'N', '5', 'D', '7', '9', '4', 'R', 'P', 'Y', 'K'}

Comparing specifications for part number: 240140299S
Differences found:
  Specifications only in Excel: {'X', 'A', 'H', 'W', '1', 'T', 'S', '6', 'F', '2', 'G', '8', 'B', '3', 'O', 'I', 'V', 'L', '/', '0', 'C', 'E', 'N', ',', '5', 'D', '7', '9', '4', 'R', 'P', 'K'}
  Specifications only in PDF: {'NOVAD', 'CHGS3', 'NHTS0', 'LSLNW', 'RPAFW', 'HTS02', 'NOTCG', 'V2L00', 'CLKS0', 'ITP15', 'LSLFW', 'RCDEV', 'CLK0

In [22]:
# prompt: guardar los resultados en un archivo de texto

# ... (Your existing code)

# Save the differences to a text file
output_filename = "Analsis_DEVO.txt"

with open(output_filename, "w") as f:
    f.write("Differences after dismissal:\n")
    for part_number, diff_data in dismissed_differences.items():
        f.write(f"Part Number: {part_number}\n")
        for data in diff_data:
            f.write(f"  Source: {data['source']}\n")
            f.write(f"  Specs: {', '.join(data.get('specs', []))}\n")
            if 'missing' in data:
                f.write(f"  Missing in {data['source']}: {', '.join(data['missing'])}\n")
            if 'extra' in data:
                f.write(f"  Extra in {data['source']}: {', '.join(data['extra'])}\n")
            f.write("-" * 20 + "\n")

print(f"Differences saved to {output_filename}")


Differences saved to Analsis_DEVO.txt
